In [ ]:
'''
    动态网页爬取：
    1、https://www.ptpress.com.cn/shopping/index
    2、经常使用 Ajax、DHTML 实现前后端数据交互
'''

import requests

url = 'https://www.ptpress.com.cn/hotBook/getHotBookList?parentTagId=75424c57-6dd7-4d1f-b6b9-8e95773c0593&rows=18&page=1'

r = requests.get(url)

dict = r.json()

print(dict)

{'data': {'total': 7853, 'rows': [{'picPath': 'https://cdn.ptpress.cn/uploadimg/Material/978-7-115-53619-8/72jpg/53619.jpg', 'resume': None, 'price': 59, 'discountPrice': '47.20', 'bookDiscount': '0.8', 'bookName': '心 稻盛和夫的一生嘱托', 'bookId': '5e6fe0f3-6ee7-40c2-8c25-9cdf9b0c87e6'}, {'picPath': 'https://cdn.ptpress.cn/uploadimg/Material/978-7-115-54342-4/72jpg/54342_s300.jpg', 'resume': None, 'price': 59.8, 'discountPrice': '47.84', 'bookDiscount': '0.8', 'bookName': '认知觉醒：开启自我改变的原动力', 'bookId': '7a745ee7-4a02-412f-942a-bf0131743346'}, {'picPath': 'https://cdn.ptpress.cn/uploadimg/Material/978-7-115-41359-8/72jpg/41359_s300.jpg', 'resume': None, 'price': 68, 'discountPrice': '54.40', 'bookDiscount': '0.8', 'bookName': '聪明的投资者（原本第4版,平装本）', 'bookId': '25c373dc-e599-4036-8534-a102aad0a776'}, {'picPath': 'https://cdn.ptpress.cn/uploadimg/Material/978-7-115-48382-9/72jpg/48382_s300.jpg', 'resume': None, 'price': 49, 'discountPrice': '39.20', 'bookDiscount': '0.8', 'bookName': '即兴演讲 掌控人生关键时刻', 

In [ ]:
import csv
import requests
import json
import pymysql

def get_one_page(url):
    header = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0'
    }

    r = requests.get(url, headers=header)
    r.raise_for_status()  # 检查请求是否成功
    return r.json()


def parse_one_page(data):
    Book_data = []

    for i in data["data"]["rows"]:
        Book_ID = i["bookId"]
        Book_name = i["bookName"]
        Book_price = i["discountPrice"]
        Bookpic_src = i["picPath"]

        Book_data.append([Book_ID, Book_name, Book_price, Bookpic_src])
        print(Book_ID, Book_name, Book_price, Bookpic_src)
    
    return Book_data  # 确保返回Book_data列表

def write_to_csv(content, path):
    with open(path, 'a', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        if f.tell() == 0:
            writer.writerow(['ID', '书名', '价格', '图片链接'])
        writer.writerows(content)

def write_to_table(content):
    pass

if __name__ == '__main__':
    url = 'https://www.ptpress.com.cn/hotBook/getHotBookList?parentTagId=75424c57-6dd7-4d1f-b6b9-8e95773c0593&rows=18&page=1'

    json_data = get_one_page(url)
    if json_data:  # 检查json_data是否为None
        content = parse_one_page(json_data)
        if content:  # 检查content是否为None
            write_to_csv(content, 'Book_data.csv')
        else:
            print("解析数据失败")
    else:
        print("获取数据失败")


In [1]:
'''
    现在我想一次性爬取所有的图书类型
    需遍历此节点？https://www.ptpress.com.cn/hotBook/getParentTagIdList
'''
import csv
import requests
import json
import pymysql

def get_one_page(url):
    header = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0'
    }

    r = requests.get(url, headers=header)
    r.raise_for_status()  # 检查请求是否成功
    return r.json()

def parse_one_page(json_data):
    
    One_theme_data = json_data["data"]
    Book_data = []
    for i in One_theme_data:

        ID = i["PARENTTAGID"]
        Name = i["PARENTTAGNAME"]
        urls = f'https://www.ptpress.com.cn/hotBook/getHotBookList?parentTagId={ID}&rows=18&page=1'
        print(ID,Name)

        r = requests.get(urls)
        r = r.text
        
        One_theme_data = json.loads(r)
        
        More_data = One_theme_data["data"]["rows"]

        

        for i in More_data:
            Book_ID = i["bookId"]
            Book_name = i["bookName"]
            Book_price = i["discountPrice"]
            Bookpic_src = i["picPath"]  

            Book_data.append([Book_ID, Book_name, Book_price, Bookpic_src])
            #print(Book_ID, Book_name, Book_price, Bookpic_src)
    
    return Book_data  # 确保返回Book_data列表
    
def write_to_csv(content, path):
    with open(path, 'a', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        if f.tell() == 0:
            writer.writerow(['ID', '书名', '价格', '图片链接'])
        writer.writerows(content)
        
def write_to_table(content):
    #1、创建数据库链接对象connect
    db = pymysql.connect(
        host='192.168.40.135',
        port=3306,
        user='root',
        passwd='123456',
        db='post_book',
        charset='utf8'
    )

    #2、获取游标对象 cursor
    cursor = db.cursor()

    #3、执行SQL语句（核心步骤：建表、增删改查）
    

    create_table = """
    create table if not exists book1(
        Book_ID varchar(100) not null primary key,
        Book_name varchar(100) not null,
        Book_price varchar(100) not null,
        Bookpic_src varchar(100) not null
    )
    """
    cursor.execute(create_table)
    db.commit()

    

    #插入数据
    insert_sql = """
    insert into book1(Book_ID,Book_name,Book_price,Bookpic_src) 
    values(%s,%s,%s,%s)
    """

    cursor.executemany(insert_sql,content)
    db.commit()

    db.rollback()
        
    #4、关闭游标和数据库连接
    cursor.close()
    db.close()
        
if __name__ == '__main__':
    
    url = 'https://www.ptpress.com.cn/hotBook/getParentTagIdList'

    json_data = get_one_page(url)

    content = parse_one_page(json_data)
    
    if json_data:  # 检查json_data是否为None
        content = parse_one_page(json_data)
        if content:  # 检查content是否为None
            write_to_csv(content, '人民邮电出版社（分类）.csv')
        else:
            print("解析数据失败")
    else:
        print("获取数据失败")

    #写入数据库
    write_to_table(content)

75424c57-6dd7-4d1f-b6b9-8e95773c0593 经管
def2b966-997b-447e-b69b-7bc84c66afcf 工业现代化
cb1ecdd6-9881-421c-bc1e-ee97687e83f2 少儿
c1a5d9df-a107-4b42-a16a-45e3a46b6e43 集邮
33af7e88-7ac9-4b91-9ff7-15e521dadc21 科普
afc45c33-b36b-45e4-81c8-059e0c6cce07 生活娱乐
4370ad12-9b5d-4850-bead-ce551a8c6d9f 机电
2a8f1030-fa2f-403a-9f38-76cfe875c184 电子
a15a734f-0ae9-41d7-9012-6ef9de2e71c8 计算机
34858549-baec-4292-b5f0-23064a7ccb6f 数学与统计学
923f77a9-018e-4872-9326-cf9c255a2a32 通信
d922af09-f90a-448b-8c8c-b54625faf31f 交通
5ba1b7ee-ccd7-4221-add5-f83fa7490e9d 外语
4b25c8b3-7d73-4504-9594-5fdbd7446915 心理学
f5ff17be-e61e-4e42-a313-ffe7dc99db75 艺术传媒
a870aadd-0094-4185-a79b-e26f7310ff67 其他
75424c57-6dd7-4d1f-b6b9-8e95773c0593 经管
def2b966-997b-447e-b69b-7bc84c66afcf 工业现代化
cb1ecdd6-9881-421c-bc1e-ee97687e83f2 少儿
c1a5d9df-a107-4b42-a16a-45e3a46b6e43 集邮
33af7e88-7ac9-4b91-9ff7-15e521dadc21 科普
afc45c33-b36b-45e4-81c8-059e0c6cce07 生活娱乐
4370ad12-9b5d-4850-bead-ce551a8c6d9f 机电
2a8f1030-fa2f-403a-9f38-76cfe875c184 电子
a15a734f-0ae9-41d7-901

In [ ]:
'''
    教师案例
'''
import csv
import requests
import json
import pymysql

#获取所有分类的ID
def get_tagID():
    url = 'https://www.ptpress.com.cn/hotBook/getParentTagIdList'
    header = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0'
    }

    r = requests.get(url, headers=header)
    r.raise_for_status()  # 检查请求是否成功
    dict_data = json.loads(r.text)
    tagID = []
    for i in dict_data['data']:
        tagID.append(["PARENTTAGID"])

    return tagID



def get_one_page(url):
    header = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0'
    }

    r = requests.get(url, headers=header)
    r.raise_for_status()  # 检查请求是否成功
    return r.json()

def parse_one_page(json_data):
    book_data = []

    dict_data = json.loads(json_data)
    book_info = dict_data['data']['rows']

    for i in book_info:
        Book_ID = i["bookId"]
        Book_name = i["bookName"]
        Book_price = i["discountPrice"]
        Bookpic_src = i["picPath"] 

        book_data.append([Book_ID,Book_name,Book_price,Bookpic_src])
    return book_data

if __name__ == '__main__':
    for type in get_tagID:
        url = f'https://www.ptpress.com.cn/hotBook/getHotBookList?parentTagId={type}&rows=18&page=1'
    json_data = get_one_page(url)
    print(json_data)